In [21]:
import numpy as np
import sys
import os
import matplotlib.pyplot as plt

from IPython.display import display, Math

sys.path.append('/home/jacob/projects/quonium/python')
from config import colr
from config import config

from mathFunc2 import doConfigCalc
import mathFunc2 as mF

conf = config()
conf.echoParams()



L : 1000
dt : 1.0
T : 0.3
tFn : 100
NY : 1
Nbb : 399999
pSampleType : 0
UniPMax : 3.0
pSampSig : 0.2
MatrElems : 0
qRGtype : 0
ECut : 40
prCut : 22
NPts : 200
ExportRates : True
NXPart : 60
NThreads : 14
HydroMode : 0
HPts : 20
doRecom : 1
doDisso : 1
RGRrateOpt : 1
Mb : 4.65
M1S : 9.091
M2S : 10.023
E1S : 0.20925
E2S : 0.723
bsig : 0.01
Ysig : 0.02
alphaS : 0.3
CF : 1.3333333333333333
NC : 3
gs : 0.75
Ch : 0
rateFile : bottom_rates_2d.tsv
boop : gleep


In [2]:
conf['StateList'] = ['1S']
conf['ChannelList'] = ['IRQ','IRG']
doConfigCalc(conf)

Constants set
IRQ_vInt1S set
IRQ_rateFv1S set
IRG_vInt1S set
IRG_rateFv1S set


In [3]:
mF.MCScInt(0, conf, mF.IRQint, '1S')

0.30929994507072694

In [18]:
def sampPrelFp1V0(conf, p1M=0):
    sN = 10000
    v1 = mF.vFp(p1M, conf['Mb'])
    vi = v1*mF.sampleSphere(sN)
    g = mF.gFv(v1)
    p1 = conf['Mb']*g*np.hstack([np.linspace(1,1,vi.shape[0]).reshape(-1, 1), vi])
    p23 = mF.sampleSphere(sN)*mF.samplefB(sN, conf)[:, np.newaxis]
    p2 = np.hstack([np.sqrt(np.power(p23[:,0],2)+np.power(p23[:,1],2)+np.power(p23[:,2],2)+np.power(conf['Mb'],2)).reshape(-1, 1), p23])
    vcm = (p1[:,1:]+p2[:,1:])/((p1[:,0]+p2[:,0])[:, np.newaxis])
    bL = mF.BoostL(vcm)
    bp1, bp2 = np.einsum('ijk,ik->ij',bL,p1), np.einsum('ijk,ik->ij',bL,p2)
    cmBpr = np.linalg.norm((1/2)*(bp1 - bp2)[:,1:],axis=1)
    vcmM = np.linalg.norm(vcm,axis=1)*0
    return [vcmM, cmBpr]

def NintRecomIRQV0(conf, vcC, Rconst, st):
    vpFunc = lambda vp: mF.IRQvpFC(vp, conf, st)
    return (1/np.power(conf['L'],3))*Rconst*np.mean(vpFunc(sampPrelFp1V0(conf)))
def NintRecomIRGV0(conf, vcC, Rconst, st):
    vpFunc = lambda vp: mF.IRGvpFC(vp, conf, st)
    return (1/np.power(conf['L'],3))*Rconst*np.mean(vpFunc(sampPrelFp1V0(conf)))

In [19]:
IRQvcC = mF.MCScInt(0, conf, mF.IRQint, '1S')
IRQrateV0 = float(NintRecomIRQV0(conf, IRQvcC, conf['IRQ_C'], '1S'))
IRGvcC = mF.MCScInt(0, conf, mF.IRGint, '1S')
IRGrateV0 = float(NintRecomIRGV0(conf, IRGvcC, conf['IRG_C'], '1S'))
print('Γ_IRQ_v=0:',IRQrateV0,' [GeV]')
print('Γ_IRG_v=0:',IRGrateV0,' [GeV]')

Γ_IRQ_v=0: 2.3165106806003e-11  [GeV]
Γ_IRG_v=0: 1.7199489693610106e-11  [GeV]


In [30]:
# IRQ rate
IRQrateRaw1 = r'\Gamma_{\text{lab}}^{\text{inel,}q} = g_+ \sum_j \frac{e^{-(\pmb{x}_i -\tilde{\pmb{x}}_j)^2/(2a_B^2)}}{(2\pi a^2_B)^{3/2}} \frac{8\alpha_s^2}{9\pi^2}' 
IRQrateRaw2 = r'\int p_1 \text{d}p_1\text{d}c_1 n_F(\gamma(1+vc_1)p_1) \int p_2 \text{d}c_2 \text{d}\phi_2 [1-n_F(\gamma(1+vc_2)p_2)]'
IRQrateRaw3 = r'\frac{p_1p_2 (1+s_1s_2\cos{\phi_2}+c_1c_2)}{p_1^2 + p_2^2 -2p_1p_2(s_1s_2\cos{\phi_2}+c_1c_2)} |\langle \Psi_{\pmb{p}_{rel}}|\pmb{r}|\psi_{nl}\rangle|^2'
print('IRQ rate:')
display(Math(IRQrateRaw1))
display(Math(IRQrateRaw2))
display(Math(IRQrateRaw3))

IRGrateRaw1 = r'\Gamma_{\text{lab}}^{\text{inel,}g} = g_+ \sum_j \frac{e^{-(\pmb{x}_i -\tilde{\pmb{x}}_j)^2/(2a_B^2)}}{(2\pi a^2_B)^{3/2}} \frac{\alpha_s^2}{3\pi^2}' 
IRGrateRaw2 = r'\int q_1 \text{d}q_1\text{d}c_1 n_B(\gamma(1+vc_1)q_1) \int q_2 \text{d}c_2 \text{d}\phi_2 [1-n_B(\gamma(1+vc_2)q_2)]'
IRGrateRaw3 = r'\frac{(q_1^2+q_2^2) (1+s_1s_2\cos{\phi_2}+c_1c_2)}{(q_1 + q_2)^2 -2q_1q_2(s_1s_2\cos{\phi_2}+c_1c_2)} |\langle \Psi_{\pmb{p}_{rel}}|\pmb{r}|\psi_{nl}\rangle|^2'
print('IRG rate:')
display(Math(IRGrateRaw1))
display(Math(IRGrateRaw2))
display(Math(IRGrateRaw3))

IRQ rate:


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

IRG rate:


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [37]:
# Executing this integration
IRQrateRaw4 = r'\text{IRQ_vcC} = \int p_1 \text{d}p_1\text{d}c_1 n_F(\gamma(1+vc_1)p_1) \int p_2 \text{d}c_2 \text{d}\phi_2 [1-n_F(\gamma(1+vc_2)p_2)] \frac{p_1p_2 (1+s_1s_2\cos{\phi_2}+c_1c_2)}{p_1^2 + p_2^2 -2p_1p_2(s_1s_2\cos{\phi_2}+c_1c_2)}'
display(Math(IRQrateRaw4))
print('Monte-carlo integrate over relavant ranges for p1, p2, c1, c2, phi2')
print('')
print('IRQ_vcC =', IRQvcC)
print('')
IRGrateRaw4 = r'\text{IRG_vcC} = \int q_1 \text{d}q_1\text{d}c_1 n_B(\gamma(1+vc_1)q_1) \int q_2 \text{d}c_2 \text{d}\phi_2 [1-n_B(\gamma(1+vc_2)q_2)] \frac{(q_1 + q_2)^2 (1+s_1s_2\cos{\phi_2}+c_1c_2)}{q_1^2 + q_2^2 -2q_1q_2(s_1s_2\cos{\phi_2}+c_1c_2)}'
display(Math(IRGrateRaw4))
print('Monte-carlo integrate over relavant ranges for q1, q2, c1, c2, phi2')
print('')
print('IRG_vcC =', IRGvcC)

<IPython.core.display.Math object>

Monte-carlo integrate over relavant ranges for p1, p2, c1, c2, phi2

IRQ_vcC = 0.3235749358449477



<IPython.core.display.Math object>

Monte-carlo integrate over relavant ranges for q1, q2, c1, c2, phi2

IRG_vcC = 0.6123918652832362
